TRABALHO DISCIPLINA INTELIGÊNCIA ARTIFICIAL APLICADA À CIÊNCIA ANIMAL - CLASSIFICAÇÃO DE RAÇAS BOVINAS EM SUBESPÉCIES

Imports de pacotes do Python

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Caminhos principais
base_path_taurinos = '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/'
source_dir = os.path.join(base_path_taurinos, 'Cattle/Taurinos')
target_base_dir = os.path.join(base_path_taurinos, 'Images/Taurinos')

# Proporções para train, validation e test
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Diretórios de destino
train_dir = os.path.join(target_base_dir, 'train')
validation_dir = os.path.join(target_base_dir, 'validation')
test_dir = os.path.join(target_base_dir, 'test')

# Criar diretórios de destino, se não existirem
for directory in [train_dir, validation_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Listar todas as imagens no diretório de origem
images = [img for img in os.listdir(source_dir) if img.lower().endswith(('png', 'jpg', 'jpeg'))]

# Garantir que há imagens suficientes
if len(images) < 3:
    raise ValueError("O diretório de origem não contém imagens suficientes para dividir.")

# Divisão em treino, validação e teste
train_files, temp_files = train_test_split(images, train_size=train_ratio, random_state=42)
validation_files, test_files = train_test_split(temp_files, test_size=test_ratio / (validation_ratio + test_ratio), random_state=42)

# Função para mover arquivos para o diretório apropriado
def move_files(file_list, destination_dir):
    for file_name in file_list:
        source_path = os.path.join(source_dir, file_name)
        target_path = os.path.join(destination_dir, file_name)
        shutil.move(source_path, target_path)
        print(f"Arquivo '{file_name}' movido para '{destination_dir}'")

# Mover arquivos para os respectivos diretórios
move_files(train_files, train_dir)
move_files(validation_files, validation_dir)
move_files(test_files, test_dir)

print("Particionamento concluído.")

print("Treino:", len(train_files))
print("Validação:", len(validation_files))
print("Teste:", len(test_files))

Arquivo 'new2-1850.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'simmental_new5-1220.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'charolais_new2-83.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new3-399.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new4-1196.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new4-1248.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new4-959.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new3-737.jpg' movido para '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/Taurinos/train'
Arquivo 'holstein_new4-1382.jpg' movido para

In [ ]:
import sys
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os


# NO CASO ESTÁ IMPORTANDO A CNN VGG16. CONSULTA NA INTERNET tensorflow.keras.applications
# POIS HÁ VARIAS OUTRAS CNNS QUE VC PODE TESTAR.
from tensorflow.keras.applications import VGG16, VGG19, ResNet152V2, DenseNet121, EfficientNetB0, EfficientNetB7


Monta o Google Drive

In [ ]:
# define cnn model using VGG16
def define_model(num_classes, input_shape):

    # PARA TESTAR OUTRAS CNNS, ALÉM DA ALTERAÇÃO DA IMPORTAÇÃO DESSA ACIMA,
    # VC VAI TROCAR VGG16 PELO NOME DA CNN IMPORTADA

    # Load the VGG16 model, excluding the top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Create a new model
    model = Sequential()
    # Add the VGG16 base model
    model.add(base_model)
    # Flatten the output from the base model
    model.add(Flatten())
    # Add new fully connected layers
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    # Output layer with num_classes units and sigmoid activation
    model.add(Dense(2, activation='softmax'))  # Saída: 1 neurônio
    #model.add(Dense(88, activation='softmax'))

    # Compile the model with categorical crossentropy for multi-class classification
    opt = SGD(learning_rate=0.001, momentum=0.9)
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

# Funcao principal
def funcao_principal(num_classes, base_path, input_shape):
    # chama a função que define o modelo da rede
    model = define_model(num_classes, input_shape)

    # cria o gerador de dados de treino fazendo a normalização dos valores de pixels na escala real [0,1] e tirando 30% das amostras para validacao
    datagen_train = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.15)

    # cria o gerador de dados de teste fazendo a normalização dos valores de pixels na escala real [0,1]
    datagen_test = ImageDataGenerator(rescale=1.0/255.0)

    # prepara os iteradores
    batch_size = 32

    # SE VOCÊ TIVER COM CONTA COLAB PRO, O CERTO É ZIPAR, BAIXAR E DESCOMPACTAR OS ARQUIVOS PARA A MAQUINA.
    # PELO Drive O ACESSO É LENTO
    train_it = datagen_train.flow_from_directory(os.path.join(base_path, 'train'),
        class_mode='categorical', batch_size=batch_size, target_size=(200, 200))

    validation_it = datagen_train.flow_from_directory(os.path.join(base_path, 'validation'),
        class_mode='categorical', batch_size=batch_size, target_size=(200, 200))

    test_it = datagen_test.flow_from_directory(os.path.join(base_path,'test'),
        class_mode='categorical', target_size=(200, 200))

    # fit model
    history = model.fit(train_it, validation_data=validation_it,
        validation_steps=validation_it.samples//batch_size, epochs=15, verbose=2)

Chama a função principal

In [ ]:
# Aciona o programa
input_shape = (200, 200, 3)
epochs = 15


base_path  = '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/'


nclasses = len([name for name in os.listdir(os.path.join(base_path, 'train')) if os.path.isdir(os.path.join(base_path, 'train'))])

print(nclasses)

funcao_principal(nclasses, base_path, input_shape)


2
Found 463 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 98 images belonging to 2 classes.
Epoch 1/15
15/15 - 259s - 17s/step - accuracy: 0.5227 - loss: 0.7502 - val_accuracy: 0.6042 - val_loss: 0.6429
Epoch 2/15
15/15 - 278s - 19s/step - accuracy: 0.5616 - loss: 0.6552 - val_accuracy: 1.0000 - val_loss: 0.4520
Epoch 3/15
15/15 - 302s - 20s/step - accuracy: 0.6890 - loss: 0.5835 - val_accuracy: 0.7396 - val_loss: 0.5738
Epoch 4/15
15/15 - 218s - 15s/step - accuracy: 0.6782 - loss: 0.5874 - val_accuracy: 0.7143 - val_loss: 0.6782
Epoch 5/15


In [ ]:
##Se já treinou o modelo e o salvou como vgg16_cattle.keras, pode carregá-lo assim:

from tensorflow.keras.models import load_model

# Carregar o modelo salvo
model = load_model("/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/vgg16_cattle.keras")

# Exibir um resumo do modelo para verificar a arquitetura
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_73"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)          │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 200, 200, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 200, 200, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 100, 100, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 100, 100, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 100, 100, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 50, 50, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 50, 50, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 50, 50, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 50, 50, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 25, 25, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 25, 25, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 25, 25, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 25, 25, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 12, 12, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_18 (Flatten)                 │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 32)                  │         589,856 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 16,484,456 (62.88 MB)

 Trainable params: 589,922 (2.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 1,179,846 (4.50 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Caminho para salvar o modelo
checkpoint_path = "/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/vgg16_cattle.keras"

# Criar o callback
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,          # Caminho do arquivo
    monitor='val_loss',                # Métrica a ser monitorada
    save_best_only=True,               # Salva apenas o melhor modelo
    save_weights_only=False,           # Salva todo o modelo, não apenas os pesos
    mode='min',                        # Menor val_loss é melhor
    verbose=2                          # Mostra logs de salvamento
)

# Usar o callback durante o treinamento
history = model.fit(
    train_it,
    validation_data=validation_it,
    epochs=15,
    callbacks=[checkpoint]             # Adicione o callback aqui
)

In [ ]:
##Se você está usando a VGG16 com Transfer Learning, pode carregar a arquitetura base do modelo e ajustar as camadas finais para o seu problema:

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Carregar a base do modelo VGG16 pré-treinada no ImageNet
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(200, 200, 3))

# Congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas personalizadas
x = Flatten()(base_model.output)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(2, activation='softmax')(x)  # Alterar o número de classes conforme necessário

# Construir o modelo final
model = Model(inputs=base_model.input, outputs=output)

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Exibir o resumo
model.summary()
model.save("/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/vgg16_cattle.keras")

# Avaliar no conjunto de teste
loss, accuracy = model.evaluate(test_it, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 200, 200, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 200, 200, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 100, 100, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 100, 100, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 100, 100, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 50, 50, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 50, 50, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 50, 50, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 50, 50, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 25, 25, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 25, 25, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 25, 25, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 25, 25, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 12, 12, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 12, 12, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │         589,856 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 15,304,610 (58.38 MB)

 Trainable params: 589,922 (2.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

NameError: name 'test_it' is not defined

Avalia o Modelo

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

base_path = '/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/Images/'
model_name = "/content/drive/MyDrive/IA_ciencia_animal/TrabalhoIA/vgg16_cattle.keras"

# cria o gerador de dados de teste fazendo a normalização dos valores de pixels na escala real [0,1]
datagen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)

test_it = datagen_test.flow_from_directory(
    os.path.join(base_path, 'test'),
    class_mode='categorical',
    target_size=(200, 200),
    shuffle=False  # Importante para manter as classes alinhadas
)

model = keras.models.load_model(os.path.join(base_path, model_name))

# Avalia o modelo
loss, accuracy, precision, recall, f1_score = model.evaluate(test_it, verbose=2)

print(f"Perda: {loss:.3f}")
print(f"Acurácia: {accuracy * 100.0:.3f}%")
print(f"Precisão: {precision * 100.0:.3f}")
print(f"Revocação: {recall * 100.0:.3f}")

# Obter as previsões do modelo
predictions = model.predict(test_it, verbose=2)

# A classe verdadeira de cada imagem
true_classes = test_it.classes

# Obter as previsões como uma classe, ou seja, o índice da classe com a maior probabilidade
predicted_classes = np.argmax(predictions, axis=1)

# Calcular a matriz de confusão com TensorFlow
conf_matrix_tf = tf.math.confusion_matrix(
    labels=true_classes,
    predictions=predicted_classes
).numpy()

print(conf_matrix_tf)

# Mostrar a matriz de confusão com o seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_tf, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_it.class_indices.keys(),
            yticklabels=test_it.class_indices.keys())
plt.title('Matriz de Confusão (TensorFlow)')
plt.xlabel('Classe Predita')
plt.ylabel('Classe Verdadeira')
plt.show()

Found 110 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 - 51s - 13s/step - accuracy: 0.5273 - loss: 0.7554


ValueError: not enough values to unpack (expected 5, got 2)